## FINAL REPORT: Predicted Chest Pain Type in Cleveland, Ohio 
(Section 002, Group 31)

## Introduction

**Background:**
    Heart disease is the leading cause of death in the United States, including the city of Cleveland in Ohio. Factors that affect the probability of contracting heart disease include, unhealthy diet, high blood pressure, high cholesterol, minimal physical activity, etc. (“Heart disease,” 2022). While heart disease symptoms depend on the type of heart disease, many common symptoms include chest pain, shortness of breath, and pain in the neck, jaw, and throat. As noted, chest pain is often associated with heart disease. Particularly angina is a chest pain caused by poor blood flow to the heart due to the buildup of thick plaques on the inner walls of the arteries carrying blood to the heart, thereby restricting the heart’s supply of blood (“Chest pain,” 2023). Several genetic and lifestyle factors contribute to the risk of developing angina. For example, there is evidence of a positive relationship between age and developing angina; an increase of plaque build up in the arteries is correlated with an increase in age (“Causes and risk,” n.d.).  

**Question:** Is it possible to predict chest pain type (1=typical angina, 2=atypical angina, 3=non-anginal pain or 4=asymptomatic*) for a given heart disease patient based on various revelent medical factors and patient information?

**Dataset:** Cleveland Heart Disease Processed Data set from https://archive.ics.uci.edu/ml/datasets/Heart+Disease. The sample size (ie. number of rows) of this dataset is 303, and there are 14 columns. The data will be further described throughout our preliminary and exploratory data analysis. 

## Methods
### Explanotory Data Analysis
Here, we will read, tidy, summarize, and visualize the dataset. Ultimately in this section, we want to find our variables of interest to use as predictors for our classification model.

First, we will start by reading in the dataset from its URL code. The data is untidy - there are no column names, and some variables that should be categorical are categorized as numerical variables instead. We must tidy the data before continuing further data analysis. Our tidy data has proper readable column names, and the variables are correctly categorized (ie. categorical values are listed numerically), which may be unhelpful when trying to visualize and understand the data in R. It is much easier to do data analysis and build a classification model from tidy data. 


We will rename the columns so that they are named as the correct variable as listed in the Cleveland Heart Disease Dataset description. Then, since we are not using all the variables given in the dataset, we will select specific columns based on our variables of interest to clean and wrangle the data further. For instance, some columns are unspecified or don't have enough information on their meaning from the website (thal, oldpeak, slope, ca, and num), and when tested further there proved to be no significant relationship that would deem them as possible predictor variables in predicting chest pain type and so we decided not include them in our data analysis. In addition, `slope` and `ca` contain question marks(?) which designate them as character types rather than integers, and we will therefore exclude them from consideration. Next, we will mutate certain variables (cp, sex, and exang)and change the type of character they were listed as, so that they are categorical and not numerical. 

#### Attach necessary libraries
The following code loads five different R packages which helps us with various tasks in our data analysis, visualization, and modeling.

In [22]:
library(tidyverse)
library(repr)
library(tidymodels)
library(RColorBrewer)
library(gridExtra)

ERROR: Error in library(gridExtra): there is no package called ‘gridExtra’


#### Import the dataset
The dataset is imported using the `read_csv` function from the R package we attached above.

In [ ]:
#read data
#add names to columns based on information from website
#clean and wrangle data
#select columns within the dataset corresponding with our selected variables of interest 
#mutate certain rows to change the type of character a variable is listed as 

data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",
                           col_names = FALSE)

tail(data)

*figure1*

As illustrated in Figure 1, the dataset lacks column names and has missing data in the `X12` column, which is denoted by a question mark '?'. Additionally, some categorical values are incorrectly labeled as `dbl` and need to be properly tidied.

#### Clean and Wrangle the data
First we rename the columns in the dataset by using more descriptive names, and stores the resulting dataset in a new variable `heart_data`."

In [ ]:
heart_data <- data |> 
        rename(
            age = X1,
            sex = X2,
            cp = X3,
            restbp = X4,
            chol = X5,
            fbs = X6, 
            restecg = X7,
            thalach = X8,
            exang = X9,
            oldpeak = X10,
            slope = X11,
            ca = X12,
            thal = X13,
            num = X14) 

head(heart_data)

*figure 2*

We will not use all the variables to answer our research question. Below we will select the specific columns based on our variables of interest to clean and wrangle the data. For instance, some columns are unspecified or don't have enough information on their meaning from the website (`thal`, `oldpeak`, `slope`, `ca`, and `num`), so we will not include them. In addition, `slope` and `ca` contain question marks(?) which designate them as character types rather than integers, and we will therefore exclude them from consideration. Some columns are untidy (categorical values are listed numerically, which may be unhelpful when trying to visualize and understand the data). This can make it difficult to visualize and interpret the data. 

Then, we create a new dataset `filter_data` by performing the following operations on the `heart_data` dataset.

In [ ]:
filter_data <- heart_data|>
    select(cp, age, restbp, chol, thalach, sex, exang)|>
    mutate(cp = as_factor(cp)) |>
    mutate(sex = as_factor(sex)) |>
    mutate(exang = as_factor(exang))
head(filter_data)

*figure3*

Now the resulting `filter_data` dataset contains a subset of columns from the original dataset, with some of the columns converted to factor variables for proper statistical analysis

#### Creating the train/test split
Since splitting the data is a randomized process, we used the function `set.seed` to ensure reproducibility. We use `initial_split` to allocate 75% of the data for the training model and the remaining 25% for testing. The 75/25 split ensures there is a representative and sizable subset of the data for both capturing the characteristics of the data and evaluating the model's performance. In testing the performance of the classifier on new unseen data, we can ensure that the model is reliable and accurate in predicting chest pain type and can be generalize to new data.

In [ ]:
#split data into training and testing, with 75% of data as training.
set.seed(2000)

heart_split <- initial_split(filter_data, prop = 0.75, strata=cp)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)

#### Summarize the data
Here we calculate the count and percentage of each chest pain type, as well as the mean and median of age, resting blood pressure, cholesterol, and maximum heart rate. They are divided into three tables, each creating a new data frame by performing different operations for each chain type.

In [ ]:
#using training data, summarize data into tables of count, mean values, and median values
count_cp <- heart_train|>
    group_by(cp)|>
    summarize(count = n()) |>
    mutate(percent = 100*count/nrow(heart_train))
count_cp

*figure4*

In [ ]:
mean_variables <- heart_train |>
    group_by(cp) |>
    summarize(mean_age=mean(age, na.rm=TRUE),
             mean_restbp=mean(restbp, na.rm=TRUE),
             mean_chol=mean(chol, na.rm=TRUE),
             mean_thalach=mean(thalach, na.rm=TRUE))
mean_variables

*figure5*

In [ ]:
median_variables <- heart_train |>
    group_by(cp) |>
    summarize(median_age=median(age, na.rm=TRUE),
             median_restbp=median(restbp, na.rm=TRUE),
             median_chol=median(chol, na.rm=TRUE),
             median_thalach=median(thalach, na.rm=TRUE))
median_variables

*figure6*

Based on our summary data above, we can interpret some of the relationships and trends between `cp` and the variables of interest. We can see that a majority of our data contains points for `cp` = 4, making asymptomic angina the most common type of angina within our dataset. 

#### Visualize the predictors and target variable

In [ ]:
#using training data, visualize numeric predictors against each other to see distribution of data

restbp_plot <- heart_train |>  
    ggplot(aes(x=age, 
               y= restbp, 
               colour = cp)) +
        labs(x = "Age (years)",
             y = "Resting Blood Pressure (mm Hg)",
            colour = 'Chest Pain Type') +
        geom_point() +
        scale_color_brewer(palette = "Set1") +
    theme(text = element_text(size = 20)) 
restbp_plot

*figure7*

In [ ]:
chol_plot <- heart_train |>  
    ggplot(aes(x=age, 
               y= chol, 
               colour = cp)) +
        labs(x = "Age (years)",
             y = "Serum cholesterol (mg/dl)",
            colour = 'Chest Pain Type') +
        geom_point() +
        scale_color_brewer(palette = "Set1") +
    theme(text = element_text(size = 20)) 
chol_plot

*figure8*

In [ ]:
thalach_plot <- heart_train |>  
    ggplot(aes(x=age, 
               y= thalach, 
               colour = cp)) +
        labs(x = "Age (years)",
             y = "Maximum Heart Rate Achieved",
            colour = 'Chest Pain Type') +
        geom_point() +
        scale_color_brewer(palette = "Set1") +
    theme(text = element_text(size = 20)) 
thalach_plot

*figure9*

As we can see, none of the plots really show a clear, distinct relationship between our target variable chest pain type and the tested predictors. This could be for many possible reasons. We only used numeric predictors - perhaps other predictors, such as sex, could yield a more clear distribution. In testing out the relationship between more variables, we could find a better predictor, or if any variable is a good predictor for chest pain at all. If results contain to produce no relationship then perhaps our target variable may need to be refined and could instead be generalized in predicting simply whether or not there is a heart disease diagnosis based on the chosen predictor variables.

In [ ]:
library(GGally)
library(broom)

In [ ]:
options(repr.plot.height = 15, repr.plot.width = 15)

heart_pairs <- filter_data |>
    ggpairs(mapping=ggplot2::aes(colour = cp))
heart_pairs

*figure10*

As we can see from the visualization above of each variable (from the filtered data) compared against each other, none of them show entirely clear relationships, with regard to chest pain type. However, it seems that `age` and `thalach` (maximum heart rate) seem to shown some level of distinction with chest pain types (compared to the other plots). In general, the purple data points (4) are around the bottom, the blue (3) at the top, and the red and green (1 and 2) are moderately scattered throughout. The other plots are completely randomly scattered. Thus, we will use these variables as predictors for our classification model, however, it is likely that our classification model will not have a high accuracy.


In addition, from previous literature, it seems that age and maximum heart rate are relevant factors relating to chest pain type. (more literature review)

Now that we have decided to use `age` and `thalach` as the predictors for our classification model, we can move on to our planned data analysis. We will start with creating a recipe, a knn specification model (using the `tune` function to determine what $k$ value to use, and our classifier with the training data set. Then, we will perform a cross-fold validation and test the metrics with $k$ values from 2-20, and plot the data to find our ideal $k$ value.

In [ ]:
set.seed(1234) 

heart_recipe <- recipe(cp ~ age + thalach, data = heart_train)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
       set_engine("kknn") |>
       set_mode("classification")

heart_vfold <- vfold_cv(heart_train, v = 5, strata = cp)

gridvals <- tibble(neighbors = seq(2, 20, 1))

heart_workflow <- workflow() |>
       add_recipe(heart_recipe) |>
       add_model(knn_spec) |>
       tune_grid(resamples = heart_vfold, grid = gridvals) |>
       collect_metrics()

accuracies <- heart_workflow |>
    filter(.metric == "accuracy")

cross_val_plot <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate") +
       scale_x_continuous(breaks = seq(0, 20, by = 1)) +  
       scale_y_continuous(limits = c(0, 1.0)) 
cross_val_plot

*figure11*

As we can see from the above plot, $k$=10 would be the best possible $k$ value, since its accuracy estimate is the highest among all the values plotted. We will now input that value into a new knn specification model and classifier.

In [ ]:
set.seed(9999) 

new_knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 10) |>
       set_engine("kknn") |>
       set_mode("classification")

heart_fit <- workflow() |>
       add_recipe(heart_recipe) |>
       add_model(new_knn_spec) |>
       fit(data = heart_train)

Now that we have our final model, we can predict on the test dataset, and report the accuracy and confusion matrix of this prediction.

In [ ]:
heart_predictions <- predict(heart_fit, heart_test) |>
    bind_cols(heart_test)
heart_predictions

heart_metrics <- heart_predictions |>
    metrics(truth = cp, estimate = .pred_class)
heart_metrics

heart_conf_mat <- heart_predictions |>
    conf_mat(truth = cp, estimate = .pred_class)
head(heart_conf_mat)

*figure12*

## Discussion

**Our Findings:**
We wanted to discover if it's possible to predict chest pain type for a given heart disease patient based on various revelent medical factors and patient information. Through our data analysis, we found that `age` and `thalach` (or maximum heart rate) would be the most likely medical factors to help with this prediction. After collecting metrics on our classification model, we found the accuracy to be 0.47. Thus, we can infer that it is possible to predict chest pain type, though the accuracy is not significantly high. We can assume there are many factors at play contributing to chest pain type, and it is difficult to predict what kind a patient may have.

**What we expected to find:**
We wanted to predict the type of chest pain (1=typical angina, 2=atypical angina, 3=non-anginal pain or 4=asymptomatic*) a patient with heart disease/conditions could expect to have based on relevent medical factors and personal information as noted by the variables of interest listed above. Since the graphs in our preliminary analysis did not show a clear distribution of data or a distinct relationship correlated with chest pain type, it was hard to determine what we would expect to find. Thus, we focused on refining our list of chosen predictor variables to determine whether the variables chosen are effective predictors or not. 
Our findings were (expected/not expected).

**Impact of our findings:**
Predicting chest pain type and therefore the severity of the chest pain can provide a physician with the necessary information needed to treat a patient's heart disease. Further data analysis could describe which factor(s), if any, directly lead to a certain chest pain type in order to help physicians when prescribing medications or choosing a course of action for a patient.

**Future Questions:**
- Is this classification model consistent with data in other parts of the world (eg. Hungary, Switzerland, etc)? 
- What other variables could be used in predicting chest pain that we haven't already accounted for, and if those variables were added, would that increase the estimate accuracy?
- Does there exist any diseases sharing the same predictor variables as chest pain? If so, how can we distinguish between these diseases and chest pain?

## References

Centers for Disease Control and Prevention. (2022, August 12). Heart disease deaths - health, United States. Centers for Disease Control and Prevention. Retrieved March 11, 2023, from https://www.cdc.gov/nchs/hus/topics/heart-disease-deaths.htm#:~:text=Heart%20disease%20has%20been%20the,excessive%20alcohol%20use%20(2)

Mayo Foundation for Medical Education and Research. (2023, March 3). Chest pain. Mayo Clinic. Retrieved March 11, 2023, from https://www.mayoclinic.org/diseases-conditions/chest-pain/symptoms-causes/syc-20370838

U.S. Department of Health and Human Services. (2022, March 24). Causes and risk factors. National Heart Lung and Blood Institute. Retrieved March 11, 2023, from https://www.nhlbi.nih.gov/health/angina/causes
